## Day 7: No Space Left On Device

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day07/puzzle.ipynb)



### Parsing

In [39]:
#!value --name sampleRaw
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k

Formally, the given input is not a list of commands. It is commands plus their output when applicable. But in order to simplify the parsing process let us consider the lines of the `ls` command output to be separate commands, as they are distinct in format.

In [40]:
type Command = 
    | CDRoot
    | CDUp
    | CD of string
    | Ls
    | Dir of string
    | File of int64*string

In [41]:
#r "nuget:Farkle, 6.3.2"
open Farkle
open Farkle.Builder
open Farkle.Builder.Regex

#load "../common/common.fsx"

Installed Packages Farkle, 6.3.2

In [42]:
let private name = 
    [
        chars PredefinedSets.AllLetters
        char '.'
    ] |> choice
    |> atLeast 1
    |> terminal "Name" (T(fun _ x -> x.ToString()))
let private number = Terminals.int64 "Number"
let private command = "Command" ||= [
    !& "$ cd /" =% CDRoot
    !& "$ cd .." =% CDUp
    !& "$ ls" =% Ls
    !& "$ cd" .>>. name => fun dirName -> CD dirName
    !& "dir" .>>. name => fun dirName -> Dir dirName
    !@ number .>>. name => fun size fileName -> File (size,fileName)
]
let private parser = RuntimeFarkle.build command

let private nameParser = RuntimeFarkle.build name

let parseCommand s = 
    s
    |> RuntimeFarkle.parseString parser
    |> Result.get


In [43]:
#!share sampleRaw --from value

let sampleCommands = sampleRaw |> Pattern1.read parseCommand
sampleCommands

index,type,Item,Item1,Item2
0,FSI_0058+Command,,,
1,FSI_0058+Command,,,
2,FSI_0058+Command+Dir,a,,
3,FSI_0058+Command+File,,14848514,b.txt
4,FSI_0058+Command+File,,8504156,c.dat
5,FSI_0058+Command+Dir,d,,
6,FSI_0058+Command+CD,a,,
7,FSI_0058+Command,,,
8,FSI_0058+Command+Dir,e,,
9,FSI_0058+Command+File,,29116,f


In [44]:
#!value --name actualRaw --from-file ./data_actual.txt

In [45]:
#!share actualRaw --from value
let actualCommands = actualRaw |> Pattern1.read parseCommand

### Part 1

We always start from the root of the filesystem, which is rather helpful. The filesystem can be described in the recursive type called `Dir`. Unfortunately, it has to allow to change values of some of its properties, because we will fill it up as we know more and more about the filesystem from the input.

In [46]:
type File = { Name: string; Size: int64 }
type Dir =
    { Name: string
      Nested: Dir list ref
      Files: File list ref
      TotalSize: int64 ref }


To solve the problem, we will make sure we maintain the `TotalSize` property in all `Dir` nodes in te tree as we read the input. To achieve this, we introduce the path from the root to the current `Dir`.
1. For each `cd x` command a new `Dir` will be prepended to the path.
2. Similarly, each `cd ..` will throw away the head of the path.
3. Every discovered file's size will be added to all the nodes in the path.

In [47]:
let execute (commands : seq<Command>) = 
    let mutable root = 
        { Name = "/"
          Nested = ref []
          Files = ref []
          TotalSize = ref 0L }
    let mutable path = [root]
    for command in commands do
        match command with
        | CDRoot -> path <- [root]
        | Ls -> ()
        | Dir _ -> ()
        | CDUp -> path <- List.tail path
        | CD dirName -> 
            let current::_ = path
            if (current.Nested.Value
                |> List.tryFind (fun d -> d.Name = dirName)
                |> Option.isSome) then
                ()

            let newDir =
                { Name = dirName
                  Nested = ref []
                  Files = ref []
                  TotalSize = ref 0L }

            current.Nested.Value <- newDir :: current.Nested.Value
            path <- newDir::path
        | File (fileSize, fileName) -> 
            let current::_ = path
            if (current.Nested.Value
                |> List.tryFind (fun d -> d.Name = fileName)
                |> Option.isSome) then
                ()

            let newFile =
                { Name = fileName
                  Size = fileSize }
                
            path
            |> List.iter (fun dir -> 
                    dir.TotalSize.Value <- dir.TotalSize.Value + newFile.Size
                )

            current.Files.Value <- newFile :: current.Files.Value
    root


In [48]:
Formatter.SetPreferredMimeTypesFor(typedefof<Dir>, "text/plain")
let rec private formatDir (sb : StringBuilder) (dir : Dir) depth = 
    sb.Append(String.replicate (depth * 2) " ")
      .Append($"- {dir.Name} (dir, totalsize={dir.TotalSize.Value})")
      .AppendLine()
      |> ignore
    dir.Nested.Value
    |> Seq.sortBy (fun nested -> nested.Name)
    |> Seq.iter (fun nested -> formatDir sb nested (depth+1))
    dir.Files.Value
    |> Seq.sortBy (fun file -> file.Name)
    |> Seq.iter (fun file -> 
        sb.Append(String.replicate (depth * 2 + 2) " ")
          .AppendLine($"- {file.Name} (file, size={file.Size})")
        |> ignore
    )
Formatter.Register<Dir>((fun x -> 
    let sb = StringBuilder()
    formatDir sb x 0
    sb.ToString()
), "text/plain")

In [49]:
let sampleFs = execute sampleCommands
sampleFs

- / (dir, totalsize=48381165)
  - a (dir, totalsize=94853)
    - e (dir, totalsize=584)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - d (dir, totalsize=24933642)
    - d.ext (file, size=5626152)
    - d.log (file, size=8033020)
    - j (file, size=4060174)
    - k (file, size=7214296)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)


Now we have a decent image of the filesystem as a tree and can do with it whatever is needed to be done.

In [50]:
let rec collectDirs (root: Dir) =
    root :: (root.Nested.Value |> List.collect collectDirs)

let sumBelow100k (root: Dir) =     
    collectDirs root
    |> Seq.map (fun x -> x.TotalSize.Value)
    |> Seq.filter (fun x -> x <= 100000)
    |> Seq.sum

sampleFs
|> sumBelow100k

95437

For the actual data:

In [51]:
let actualFs = execute actualCommands
actualFs |> sumBelow100k

1307902

### Part 2

In [52]:
let directoryToDelete (root : Dir) = 
    let total = 70000000L
    let required = 30000000L
    let unused = total - root.TotalSize.Value
    let toFree = required - unused
    
    collectDirs root
    |> List.filter (fun x -> x.TotalSize.Value >= toFree)
    |> List.minBy (fun x -> x.TotalSize.Value)

In [53]:
sampleFs
|> directoryToDelete
|> displayPipe
|> (fun x -> x.TotalSize.Value)

- d (dir, totalsize=24933642)
  - d.ext (file, size=5626152)
  - d.log (file, size=8033020)
  - j (file, size=4060174)
  - k (file, size=7214296)


24933642

For the actual data:

In [54]:
actualFs
|> directoryToDelete
|> (fun x -> x.TotalSize.Value)

7068748